In [2]:
%load_ext cython
%matplotlib inline
import sys
sys.path.append('/home/bram/ANTS')
sys.path.append('/home/bram/ANTS/entropy')
for p in sys.path:
    print(p)


/usr/lib/python36.zip
/usr/lib/python3.6
/usr/lib/python3.6/lib-dynload
/home/bram/PythonEnvs/Ants/lib/python3.6/site-packages
/home/bram/PythonEnvs/Ants/lib/python3.6/site-packages/IPython/extensions
/home/bram/.ipython
/home/bram/ANTS
/home/bram/ANTS/entropy


In [ ]:
%%cython -a

from cythonic.plugins.positions cimport point
from cythonic.plugins.functions cimport transform
from libc.math cimport M_PI as PI
cimport cython

cdef struct observations:
    double lft
    double rght
    
cdef struct AntState:
    # " position and orientation "
    unsigned int id
    point pos #{x,y} in mm
    point left #left sensor location
    point right #right sensor location
    double theta #azimuth in degrees
    double omega #angular rotation in degrees/second
    double v # mm/s
    observations Q_obs
    
    #" ant status "
    bint foodbound
    bint active
    bint out_of_bounds
    
    #" individual timers "
    double rng_timer # for the random number generator
    double time #time since last event
    
cdef AntState X
X.id = 1

cdef struct fun_args:
    double gain
    double breakpoint
    
# function type for observe function
ctypedef void (*f_obs)(AntState*,fun_args*, observations*)

@cython.cdivision(True)
@cython.wraparound(False)
@cython.boundscheck(False)
@cython.nonecheck(False)
@cython.initializedcheck(False)
cdef void observe_linear(AntState* s, fun_args* a, observations* Q):
    s[0].Q_obs.lft = Q[0].lft*a[0].gain
    s[0].Q_obs.rght = Q[0].rght*a[0].gain

@cython.cdivision(True)
@cython.wraparound(False)
@cython.boundscheck(False)
@cython.nonecheck(False)
@cython.initializedcheck(False)
cdef class Ant:
    cdef AntState* state
    cdef:
        # " geometric properties "
        double l
        double sens_offset
        
        # sensor properties
        double gain
        f_obs sens_fun
        fun_args obs_fun_args
        
        
        # actuator properties
        double drop_quantity
        double return_factor
        double drop_beta
    
    def __cinit__(self, double l, double sens_offset, double gain, double drop_quantity, double return_factor,
                 double drop_beta):
        self.l = l
        self.sens_offset = sens_offset
        self.gain = gain
        self.drop_quantity = drop_quantity
        self.return_factor = return_factor
        self.drop_beta = drop_beta
        self.sens_fun = observe_linear
        self.obs_fun_args.gain = 1
        
    cdef void rotate(self,double* dt):
        " compute the angular speed (degrees/second) based on sensing "
        self.state[0].omega = self.gain*180./PI*(
            self.state[0].Q_obs.lft-self.state[0].Q_obs.rght)
        self.increase_azimuth(dt)
        
    cdef void gradient_step(self, double *dt, observations Q):
        " execute the sequence for differential based stepping "
        " sniff pheromone -> rotate -> step -> update sensors "
        
        self.observe(*Q)
        self.rotate(dt)
        self.step(dt)
        self.set_sensors()
        
    cdef void observe(self, observations* Q):
        # fill Q_obs(lft, right)
        self.sens_fun(self.state, &self.obs_fun_args, Q)
        
    cdef void step(self, double * dt):
        " do a step in the current direction ,do boundary check as well "
        self.state[0].time += dt[0] # update internal timer
        
        # update position
        cdef double step_size = self.state[0].v*dt[0]
        self.state[0].pos = transform(self.state[0].theta, &step_size, &self.state.pos)
        
    cdef void out_of_bounds(self, bint oob):
        " toggle out of bounds status "
        self.state[0].out_of_bounds = oob
        
    cdef void set_sensors(self):
        " calculate the position of the left and right sensor antennas "
        self.state[0].left = transform(self.state[0].theta + self.sens_offset, &self.l, &self.state.pos)
        self.state[0].right = transform(self.state[0].theta - self.sens_offset, &self.l, &self.state.pos)
        
    cdef void activate(self):
        " mark the ant as active "
        self.state[0].active = True
    
    cdef void increase_azimuth(self, double * dt):
        " ensure azimuth stays within [0,360) interval "
        " increase azimuth based on the angular speed and the time interval "
        self.state[0].theta = (self.state[0].theta+self.state[0].omega*dt[0])%360

        
    cdef void set_state(self,AntState* s):
        self.state = s

cdef Ant ant = Ant(l  =10., sens_offset = 45, gain = 1., drop_quantity = 1., return_factor = 1.5, drop_beta = 2.,
           )

cdef double x = 0.
cdef double dt = .1
cdef observations Q
Q.lft = 1
Q.rght = 2
X.v = 10
ant.set_state(&X)
ant.activate()
ant.set_sensors()
ant.gradient_step(&dt, Q = Q)
print(X)

In [37]:
%%cython -a
# distutils: language = c++

from libcpp.vector cimport vector

from cythonic.core.ant cimport Ant, ant_state
from cythonic.plugins.sens_structs cimport observations
cdef Ant ant = Ant(l  =10., sens_offset = 45, gain = 1., drop_quantity = 1., return_factor = 1.5, drop_beta = 2.,
           )

cdef double x = 0.
cdef double dt = .1
cdef observations Q
cdef ant_state X
X.id = 1
Q.lft = 1
Q.rght = 2
X.v = 10
ant.set_state(&X)
ant.activate()
ant.set_sensors()
ant.gradient_step(&dt, Q = &Q)
# print(X)

""" ============
    The queen is the controller of the ants (agents)
    It is encouraged to play 'Master of Puppets' while executing this class
    ============ """
cdef class Queen:
    cdef public vector[ant_state] state_list
    cdef public vector[observations] pheromone_vec
    cdef readonly Ant agent #" ant template "
    cdef:
        readonly unsigned int n
        readonly unsigned int count_active
        readonly double dt  
        
    def __cinit__(self,unsigned int n, ant_dict, double dt):
        " initialize the controller (Queen) "
        self.n = n # total number of agents
        state_list = new vector[ant_state](n) # reserve memory for a vector of states

        " sim settings "
        self.dt = dt

        " deploy a model for the agent, which accepts and modifies a state "
        self.agent = Ant(**ant_dict)

    cpdef readonly void deploy(self, ant_state s):
        " activate an ant "
        s.active = True # activate the ant
        self.state_list.push_back(s) # store the ant state
        self.count_active += 1 # count active ants

    cdef readonly void step_all(self,):
        " step all, else step specified "
        for i in range(self.count_active):
            " todo: sense on map"
            self.gradient_step(i, &self.pheromone_vec[i].data())


    cdef readonly void gradient_step(self, int ant_id, observations * Q):
        " do the gradient stepping on the agent "
        self.agent.set_state(&self.state_list[ant_id])
        self.agent.gradient_step(&self.dt, Q)


    cpdef readonly void print_pos(self):
        " print the position struct of all ant in the state vector "
        for i in range(self.state_list.size()):
            print(self.state_list[i].id,self.state_list[i].pos)

cdef Queen queen = Queen(10, {'l':10.,'sens_offset':45, 'gain':1., 'drop_quantity':1,'return_factor':1.5, 'drop_beta':2.}, dt = .1)
queen.deploy(X)
for i in range(100):
    queen.step_all()
    queen.print_pos()



Error compiling Cython file:
------------------------------------------------------------
...

    cdef readonly void step_all(self,):
        " step all, else step specified "
        for i in range(self.count_active):
            " todo: sense on map"
            self.gradient_step(i, &self.pheromone_vec[i].data())
                                                       ^
------------------------------------------------------------

/home/bram/.cache/ipython/cython/_cython_magic_8a1f6afdf0a50367045a4c65c97014f5.pyx:58:56: Object of type 'observations &' has no attribute 'data'

Error compiling Cython file:
------------------------------------------------------------
...

    cdef readonly void step_all(self,):
        " step all, else step specified "
        for i in range(self.count_active):
            " todo: sense on map"
            self.gradient_step(i, &self.pheromone_vec[i].data())
                                 ^
-----------------------------------------------------------

TypeError: object of type 'NoneType' has no len()

In [34]:
%%cython -a
# distutils: language = c++

import numpy as np
from libcpp.vector cimport vector

cdef struct observations:
    double lft
    double rght
    
cdef double f_1 = 0., f_2 = 0.

cdef observations Q
cdef vector[observations] qvec

for i in range(10):
    f_1 = np.random.rand()
    f_2 = np.random.rand()
    Q = {'lft':f_1, 'rght':f_2}
    qvec.push_back(Q)

cdef void printq(observations * QQ):
    print(QQ[0])
    
cdef void printqvec(vector[observations]* Qvec):
    cdef int i
    for i in range(Qvec[0].size()):
#         O = Qvec[i].data()[0]
        printq(Qvec[i].data())
    
for i in range(qvec.size()):
    printq(&qvec[i])


{'lft': 0.03571476395929596, 'rght': 0.1154160989202151}
{'lft': 0.4859524372461552, 'rght': 0.33334756492993833}
{'lft': 0.5369595005516425, 'rght': 0.8808758539074508}
{'lft': 0.7996772274845938, 'rght': 0.7198223338430414}
{'lft': 0.9080394905198004, 'rght': 0.5635481150421527}
{'lft': 0.45955394844057174, 'rght': 0.9458367027733525}
{'lft': 0.5478462792182733, 'rght': 0.5680607881576666}
{'lft': 0.0994249774349456, 'rght': 0.4447537910611461}
{'lft': 0.4277999901654108, 'rght': 0.5096771440122475}
{'lft': 0.36970959437312667, 'rght': 0.43219813310776645}
